## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Wed Mar 19 23:52:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   86C    P0             68W /   85W |    1524MiB /   8192MiB |     55%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\VIRAT\Projects\OCR\notebooks


## Install YOLOv8


Run the following commands in terminal to install libraries


```
pip install roboflow
pip install ultralytics
```

In [3]:
from IPython import display
display.clear_output()
from IPython.display import display, Image
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import glob
from IPython.display import Image, display

Ultralytics 8.3.93  Python-3.10.9 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Setup complete  (16 CPUs, 63.7 GB RAM, 859.2/953.0 GB disk)


## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Custom Dataset

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="D0mdk9dsanv40HGISExw")
project = rf.workspace("solar-h55md").project("solar-panel-qcdgb")
version = project.version(4)
dataset = version.download("yolov8-obb")

## Custom Training

In [ ]:
%cd {HOME}

In [ ]:
!yolo task=obb mode=train model=yolov8m-obb.pt data={dataset.location}/data.yaml epochs=75 imgsz=1200 batch=6

In [ ]:
%cd {HOME}

In [ ]:
Image(filename=f'{HOME}/runs/obb/train2/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/train2/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/train2/val_batch0_pred.jpg', width=700)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=obb mode=val model={HOME}/runs/obb/train2/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=obb mode=predict model={HOME}/runs/obb/train2/weights/best.pt conf=0.5 source={dataset.location}/test/images save=true

In [ ]:
for image_path in glob.glob(f'{HOME}/runs/obb/predict/*.jpg'):
      display(Image(filename=image_path, height=600))
      print("\n")

In [ ]:
%cd {HOME}

In [ ]:
!yolo train \
  model=/content/drive/MyDrive/ISRO/runs/obb/train2/weights/best.pt \
  data=/content/drive/MyDrive/ISRO/datasets/Solar-Panel-4/data.yaml \
  imgsz=1216 \
  epochs=150 \
  batch=8 \
  project=/content/drive/MyDrive/ISRO/runs/obb \
  name=train2.1